Import Initial Data

In [1]:
import os
# run once
# os.add_dll_directory("C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/bin")
import tensorflow as tf
# tf.config.list_physical_devices('GPU')


from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
gpus = tf.config.list_physical_devices('GPU')
print(gpus)
tf.config.set_visible_devices(gpus[0], 'GPU')



[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5466419396212756211
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5718933504
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6065668296228390391
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [79]:
# imports 
from tensorflow import keras
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [ ]:
# Figure out how to import this model 
# from tensorflow.keras.applications.resnet50v2 import ResNet50V2

In [15]:
# Note: There are only 8 images per label here, so we're going to have to change what we do here 
# Confirmed that the # of images in the 3 folders sum to 5,856, which is the total number of images 


SPLIT DATA RUN ONLY ONCE !!!!!!

In [85]:
# Split data set into train, val, and test
# just change the directory
# ONLY RUN ONCE DO NOT RUN AGAIN
import splitfolders

splitfolders.ratio("D:\Paul_Backup\paulj\Pneumonia-Classification\src\data", output = "D:\Paul_Backup\paulj\Pneumonia-Classification\src\data_split", seed = 1337, ratio = (0.6,0.2,0.2), group_prefix=None, move=False)

# add code to check for data_split directory if exists delete and create again or don't run 


In [86]:
# Training data: 3512
print(os.getcwd())
train_data = keras.utils.image_dataset_from_directory(
    'D:\\Paul_Backup\\paulj\\Pneumonia-Classification\\src\\data_split\\train',
    image_size=(180, 180), 
    batch_size=32, 
    seed = 10
)

d:\Paul_Backup\paulj\Pneumonia-Classification\src\test_notebooks\paul_test
Found 3512 files belonging to 2 classes.


In [87]:
# Validation data: 1170
val_data = keras.utils.image_dataset_from_directory(
    'D:\\Paul_Backup\\paulj\\Pneumonia-Classification\\src\\data_split\\val',
    image_size=(180, 180), 
    batch_size=32, 
    seed = 10
)

Found 1170 files belonging to 2 classes.


In [88]:
# Test data: 1174
test_data = keras.utils.image_dataset_from_directory(
    'D:\\Paul_Backup\\paulj\\Pneumonia-Classification\\src\\data_split\\test',
    image_size=(180, 180), 
    batch_size=32, 
)

Found 1174 files belonging to 2 classes.


In [89]:
fulltrain = train_data.concatenate(val_data)


In [42]:
len(list(fulltrain))

147

In [43]:
tf.data.experimental.cardinality(fulltrain)

<tf.Tensor: shape=(), dtype=int64, numpy=147>

In [45]:
tf.data.experimental.cardinality(val_data)

<tf.Tensor: shape=(), dtype=int64, numpy=37>

In [46]:
tf.data.experimental.cardinality(train_data)

<tf.Tensor: shape=(), dtype=int64, numpy=110>

Initial ResNet 50 Model:

In [80]:
# Function to build model 
def build_resnet50_model():
    
    # Define input shape for the model 
    inputs = keras.Input(shape = (180, 180, 3))
    # Resnet 50 basemodel 
    base_model = ResNet50(input_shape = (180, 180, 3), weights = 'imagenet', include_top = False)
    
    for layer in base_model.layers:
        layer.trainable = False
    
    x = keras.applications.resnet50.preprocess_input(inputs)
    x = base_model(x)
    x = keras.layers.Flatten()(x)
 
    x = keras.layers.Dense(256, activation = 'relu')(x)
    x = keras.layers.Dropout(0.2)(x)
    outputs = keras.layers.Dense(1, activation = 'sigmoid')(x)
    
    model = keras.Model(inputs, outputs)
    model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [81]:
# Build the model: 
resnet50_mod = build_resnet50_model()

In [82]:
# Function to fit model 
def fit_model(model, train_set, validation_set):
    """Fit a model with the above stated criteria"""
    # Set patience to 5 so it doesn't take too long to fit 
    early_stopping = keras.callbacks.EarlyStopping(patience = 5)
    
    model.fit(train_set, 
              validation_data = validation_set, 
              callbacks = [early_stopping], 
              epochs = 500)
    
    return model

In [83]:
# Fit the model: 
# Started 12:18 pm; ended 12:55 pm --> 37 min to train 
# Best model: 
fitted_resnet50_mod = fit_model(resnet50_mod, train_data, val_data)

Epoch 1/500
110/110 [==============================] - 10s 63ms/step - loss: 4.1018 - accuracy: 0.9052 - val_loss: 0.2027 - val_accuracy: 0.9462
Epoch 2/500
110/110 [==============================] - 6s 57ms/step - loss: 0.3226 - accuracy: 0.9462 - val_loss: 0.2598 - val_accuracy: 0.9547
Epoch 3/500
110/110 [==============================] - 6s 57ms/step - loss: 0.2105 - accuracy: 0.9655 - val_loss: 0.1572 - val_accuracy: 0.9538
Epoch 4/500
110/110 [==============================] - 6s 57ms/step - loss: 0.1836 - accuracy: 0.9690 - val_loss: 0.2464 - val_accuracy: 0.9581
Epoch 5/500
110/110 [==============================] - 6s 57ms/step - loss: 0.1050 - accuracy: 0.9778 - val_loss: 0.1235 - val_accuracy: 0.9692
Epoch 6/500
110/110 [==============================] - 6s 57ms/step - loss: 0.1078 - accuracy: 0.9795 - val_loss: 0.2403 - val_accuracy: 0.9650
Epoch 7/500
110/110 [==============================] - 6s 57ms/step - loss: 0.0806 - accuracy: 0.9860 - val_loss: 0.4862 - val_accuracy

In [84]:
fitted_resnet50_mod.evaluate(test_data)

37/37 [==============================] - 2s 40ms/step - loss: 0.2985 - accuracy: 0.9719


[0.2985367178916931, 0.9718909859657288]

Initial Inception V3 model

In [27]:
# Function to build model 
def build_inceptionV3_model():
    
    # Define input shape for the model 
    inputs = keras.Input(shape = (180, 180, 3))
    # InceptionV3 base model 
    base_model = InceptionV3(input_shape = (180, 180, 3), weights = 'imagenet', include_top = False)
    
    for layer in base_model.layers:
        layer.trainable = False
    
    x = keras.applications.resnet50.preprocess_input(inputs)
    x = base_model(x)
    x = keras.layers.Flatten()(x)
 
    x = keras.layers.Dense(256, activation = 'relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(1, activation = 'sigmoid')(x)
    
    model = keras.Model(inputs, outputs)
    model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [28]:
# Build the model: 
inceptionV3_mod = build_inceptionV3_model()

87910968/87910968 [==============================] - 3s 0us/step


In [29]:
# Fit the model: 
# Started 12:48 pm; ended pm --> 9 min to train 
fitted_inceptionV3_mod = fit_model(inceptionV3_mod, train_data, val_data)

Epoch 1/500
110/110 [==============================] - 9s 54ms/step - loss: 45.4147 - accuracy: 0.8260 - val_loss: 0.2524 - val_accuracy: 0.9162
Epoch 2/500
110/110 [==============================] - 5s 46ms/step - loss: 3.0751 - accuracy: 0.8331 - val_loss: 0.6364 - val_accuracy: 0.8940
Epoch 3/500
110/110 [==============================] - 5s 45ms/step - loss: 1.3997 - accuracy: 0.8485 - val_loss: 0.4082 - val_accuracy: 0.9051
Epoch 4/500
110/110 [==============================] - 5s 44ms/step - loss: 1.0579 - accuracy: 0.8730 - val_loss: 0.2310 - val_accuracy: 0.9385
Epoch 5/500
110/110 [==============================] - 5s 44ms/step - loss: 0.6529 - accuracy: 0.8682 - val_loss: 0.3089 - val_accuracy: 0.9239
Epoch 6/500
110/110 [==============================] - 5s 44ms/step - loss: 0.6720 - accuracy: 0.8687 - val_loss: 0.2724 - val_accuracy: 0.9419
Epoch 7/500
110/110 [==============================] - 5s 44ms/step - loss: 0.5545 - accuracy: 0.8767 - val_loss: 0.2505 - val_accuracy

In [34]:
fitted_inceptionV3_mod.evaluate(test_data)

37/37 [==============================] - 2s 37ms/step - loss: 0.6130 - accuracy: 0.9216


[0.6129802465438843, 0.9216354489326477]

In [65]:
# Function to build model 
def build_resnet50V2_model():
    
    # Define input shape for the model 
    inputs = keras.Input(shape = (180, 180, 3))
    # Resnet 50 basemodel 
    base_model = ResNet50V2(input_shape = (180, 180, 3), weights = 'imagenet', include_top = False)
    
    for layer in base_model.layers:
        layer.trainable = False
    
    x = keras.applications.resnet_v2.preprocess_input(inputs)
    x = base_model(x)
    x = keras.layers.Flatten()(x)
 
    x = keras.layers.Dense(256, activation = 'relu')(x)
    x = keras.layers.Dropout(0.2)(x)
    outputs = keras.layers.Dense(1, activation = 'sigmoid')(x)
    
    model = keras.Model(inputs, outputs)
    model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [66]:
# Build the model: 
resnet50V2_mod = build_resnet50V2_model()

In [67]:
# Fit the model: 
# Started 12:48 pm; ended pm --> 9 min to train 
resnet50V2_mod = fit_model(resnet50V2_mod, train_data, val_data)

Epoch 1/500
110/110 [==============================] - 10s 62ms/step - loss: 1.9958 - accuracy: 0.8944 - val_loss: 0.1643 - val_accuracy: 0.9265
Epoch 2/500
110/110 [==============================] - 6s 56ms/step - loss: 0.2709 - accuracy: 0.9342 - val_loss: 0.1457 - val_accuracy: 0.9556
Epoch 3/500
110/110 [==============================] - 7s 58ms/step - loss: 0.2631 - accuracy: 0.9513 - val_loss: 0.1783 - val_accuracy: 0.9504
Epoch 4/500
110/110 [==============================] - 6s 57ms/step - loss: 0.1821 - accuracy: 0.9638 - val_loss: 0.1727 - val_accuracy: 0.9547
Epoch 5/500
110/110 [==============================] - 6s 57ms/step - loss: 0.0985 - accuracy: 0.9769 - val_loss: 0.1756 - val_accuracy: 0.9564
Epoch 6/500
110/110 [==============================] - 7s 58ms/step - loss: 0.0634 - accuracy: 0.9829 - val_loss: 0.7078 - val_accuracy: 0.9239
Epoch 7/500
110/110 [==============================] - 6s 56ms/step - loss: 0.0768 - accuracy: 0.9826 - val_loss: 0.2488 - val_accuracy

In [64]:
resnet50V2_mod.evaluate(test_data)

37/37 [==============================] - 2s 39ms/step - loss: 0.5262 - accuracy: 0.9583


[0.5261532664299011, 0.9582623243331909]

In [44]:
# maybe use docker instead of pickle

import pickle
filename = 'pneumonia_inceptionV.pkl'
pickle.dump(fitted_inceptionV3_mod, open(filename, 'wb'))

INFO:tensorflow:Assets written to: ram://2f0b1158-0ad8-4ee0-8529-f38bd479d67f/assets


INFO:tensorflow:Assets written to: ram://2f0b1158-0ad8-4ee0-8529-f38bd479d67f/assets
